# 최적화

모델을 학습하는 반복 과정 속에서 모델은 출력을 추측하고, 추측과 정답 사이의 오류(손실(loss))를 계산, 매개변수에 대한 오류의 도함수(derivative)를 수집한 뒤, 경사하강법을 사용하여 이 Parameter들을 최적화(optimize)

# Preset

In [1]:
# 앞에서 했던대로 모델 구성
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# 하이퍼파라미터

**하이퍼파라미터(Hyperparameter)**는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수. 서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율(convergence rate)에 영향을 미칠 수 있다.

참고 : https://tutorials.pytorch.kr/beginner/hyperparameter_tuning_tutorial.html


학습 시에는 다음과 같은 하이퍼파라매터를 정의:
- 에폭(epoch) 수 - 데이터셋을 반복하는 횟수
- 배치 크기(batch size) - 각 에폭에서 한 번에 모델에 전달할 데이터 샘플의 수
- 학습율(learning rate) - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있다.

# 최적화 단계(Optimization Loop)

하이퍼파라미터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화할 수 있다. 최적화 단계의 각 반복(iteration)을 **에폭** 이라고 부릅니다.

하나의 에폭은 다음 두 부분으로 구성:
- 학습 단계(train loop) - 학습용 데이터셋을 반복(iterate)하고 최적의 매개변수로 수렴.


- 검증/테스트 단계(validation/test loop) - 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복(iterate).

# 손실 함수(loss function)

학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제공하지 않을 확률이 높다. 손실 함수(loss function)는 획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정, 학습 중에 이 값을 최소화하려고 함.


주어진 데이터 샘플을 입력으로 계산한 예측과 정답(label)을 비교하여 손실(loss)을 계산.

일반적인 손실함수에는 회귀 문제(regression task)에 사용하는 nn.MSELoss(평균 제곱 오차(MSE; Mean Square Error))나 분류(classification)에 사용하는 nn.NLLLoss (음의 로그 우도(Negative Log Likelihood)), 그리고 nn.LogSoftmax와 nn.NLLLoss를 합친 nn.CrossEntropyLoss 등이 있다.


모델의 출력 로짓(logit)을 nn.CrossEntropyLoss에 전달하여 로짓(logit)을 정규화하고 예측 오류를 계산

# 옵티마이저

최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정. 최적화 알고리즘은 이 과정이 수행되는 방식(여기에서는 확률적 경사하강법(SGD; Stochastic Gradient Descent))을 정의. 모든 최적화 절차(logic)는 optimizer 객체에 캡슐화(encapsulate). 여기서는 SGD 옵티마이저를 사용하고 있다.

PyTorch에는 ADAM이나 RMSProp과 같은 다른 종류의 모델과 데이터에서 더 잘 동작하는 다양한 옵티마이저가 있다.

학습하려는 모델의 매개변수와 학습율(learning rate) 하이퍼파라미터를 등록하여 옵티마이저를 초기화

https://pytorch.org/docs/stable/optim.html

학습 단계(loop)에서 최적화는 세단계:
- optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정. 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정.
- loss.backward()를 호출하여 예측 손실(prediction loss)을 역전파. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장.
- 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정

# 전체 코드

In [2]:
def train_loop(dataloader, model, loss_fn, optimizer): #학습 단계
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn): # 테스트 단계
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [3]:
learning_rate = 0.001
batch_size = 64
epochs = 20

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)

    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.291761  [    0/60000]
loss: 2.279869  [ 6400/60000]
loss: 2.273103  [12800/60000]
loss: 2.268465  [19200/60000]
loss: 2.248451  [25600/60000]
loss: 2.237311  [32000/60000]
loss: 2.245444  [38400/60000]
loss: 2.225202  [44800/60000]
loss: 2.216607  [51200/60000]
loss: 2.194067  [57600/60000]
Test Error: 
 Accuracy: 56.4%, Avg loss: 0.034412 

Epoch 2
-------------------------------
loss: 2.211507  [    0/60000]
loss: 2.181589  [ 6400/60000]
loss: 2.149193  [12800/60000]
loss: 2.153010  [19200/60000]
loss: 2.099573  [25600/60000]
loss: 2.095394  [32000/60000]
loss: 2.115808  [38400/60000]
loss: 2.072742  [44800/60000]
loss: 2.063223  [51200/60000]
loss: 2.024907  [57600/60000]
Test Error: 
 Accuracy: 58.3%, Avg loss: 0.031687 

Epoch 3
-------------------------------
loss: 2.065447  [    0/60000]
loss: 2.001112  [ 6400/60000]
loss: 1.937198  [12800/60000]
loss: 1.957265  [19200/60000]
loss: 1.847787  [25600/60000]
loss: 1.875324  [32000/600